In [9]:
from keras.applications import MobileNetV2
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation

import numpy as np
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from livelossplot import PlotLossesKeras
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense 
from keras.layers import Dense, GlobalAveragePooling2D


In [17]:
base_model = MobileNetV2(weights='imagenet', include_top=False,  input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x) 

predictions = Dense(7, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer = Adam(),
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [18]:
def preprocess_input(image):
    return image/255

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, vertical_flip = True, horizontal_flip=True, validation_split = 0.2) 

train_generator = train_datagen.flow_from_directory('./data',
    target_size=(224, 224),
    batch_size=32, class_mode='categorical', subset='training')

validation_generator = train_datagen.flow_from_directory(
    './data',  
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation')  

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 64,
    epochs = 10)
#     class_weight=
#             {0: 99,
#             1: 93,
#             2: 157,
#             3: 13, 
#             4: 119,
#             5: 136,
#             6: 101}


Found 647 images belonging to 7 classes.
Found 157 images belonging to 7 classes.
Epoch 1/10
20/20 [==============================] - 103s 5s/step - loss: 0.8293 - acc: 0.7254 - val_loss: 4.3257 - val_acc: 0.5312
Epoch 2/10
20/20 [==============================] - 108s 5s/step - loss: 0.3100 - acc: 0.9133 - val_loss: 0.9735 - val_acc: 0.8172
Epoch 3/10
 2/20 [==>...........................] - ETA: 20s - loss: 0.5036 - acc: 0.9062

KeyboardInterrupt: 

In [7]:
def check_keras_backend(print_gpu_availavility = True):
    from tensorflow.python.client import device_lib
    assert 'GPU' in str(device_lib.list_local_devices())

    from keras import backend
    assert len(backend.tensorflow_backend._get_available_gpus()) > 0

    if print_gpu_availavility:
        print('Keras: backend is GPU')
check_keras_backend()

Keras: backend is GPU


In [20]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(0.001/100),
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 64,
    epochs = 2)

Epoch 1/2
20/20 [==============================] - 103s 5s/step - loss: 0.2021 - acc: 0.9389 - val_loss: 1.1758 - val_acc: 0.8281
Epoch 2/2
10/20 [==============>...............] - ETA: 50s - loss: 0.1316 - acc: 0.9625

KeyboardInterrupt: 

In [21]:
print('True', (np.argmax(y_test[2])))
print('Predicted', np.argmax(model.predict(preprocess_input(Xtest[2]).reshape(1, 224, 224, 3))))

NameError: name 'y_test' is not defined

In [29]:
model.save("model.h5")